In [22]:
# 模块加载
import pandas as pd
import numpy as np
import jieba
import fasttext

In [23]:
# 数据加载
train = pd.read_csv('train.tsv', sep='\t', encoding='gb18030')
train.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [52]:
# 第一层第二层第三层类别转序列号
typeitems = np.array([x.split('--') for x in train['TYPE']])
train['TYPE1'] = typeitems[:,0]
train['TYPE2'] = typeitems[:,1]
train['TYPE3'] = typeitems[:,2]

type_dict = {}
for tp in train['TYPE1']:
    if tp not in type_dict.keys():
        type_dict[tp] = len(type_dict)
train['TYPE_NUM1'] = [type_dict[tp] for tp in train['TYPE1']]
print('第一层类别总数：%d' % (len(type_dict)))

type2_dict = {}
for tp in train['TYPE2']:
    if tp not in type2_dict.keys():
        type2_dict[tp] = len(type2_dict)
train['TYPE_NUM2'] = [type2_dict[tp] for tp in train['TYPE2']]
print('第二层类别总数：%d' % (len(type2_dict)))

type3_dict = {}
for tp in train['TYPE3']:
    if tp not in type3_dict.keys():
        type3_dict[tp] = len(type3_dict)
train['TYPE_NUM3'] = [type3_dict[tp] for tp in train['TYPE3']]
print('第三层类别总数：%d' % (len(type3_dict)))

train.head()

第一层类别总数：22
第二层类别总数：191
第三层类别总数：1193


,ITEM_NAME,TYPE,TYPE1,TYPE2,TYPE3,TYPE_NUM1,TYPE_NUM2,TYPE_NUM3
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值,本地生活,游戏充值,QQ充值,0,0,0
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值,本地生活,游戏充值,QQ充值,0,0,0
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值,本地生活,游戏充值,QQ充值,0,0,0
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值,本地生活,游戏充值,QQ充值,0,0,0
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值,本地生活,游戏充值,QQ充值,0,0,0


In [28]:
# 根据第一层第二层构造第二层第三层分类索引
type2_list = []
for i in range(len(type_dict)):
    a = train[train.TYPE_NUM1 == i].index.tolist()
    type2_list.append(a)

type3_list = []
for i in range(len(type2_dict)):
    a = train[train.TYPE_NUM2 == i].index.tolist()
    type3_list.append(a)

In [29]:
# 数据打乱，切分训练集和测试集
def train_test_split(x, y, seed = 10):
    np.random.seed(seed)
    shuffle_indices = np.random.permutation(np.arange(len(x)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    x_train = x_shuffled[:int(len(x)*0.95)]
    y_train = y_shuffled[:int(len(x)*0.95)]

    x_test = x_shuffled[int(len(x)*0.95):]
    y_test = y_shuffled[int(len(x)*0.95):]
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test

# 写标准fasttext标准文件
def data_write(filename, x, y):
    with open(filename, 'w') as f:
        for i, doc in enumerate(x):
            doc = jieba.cut(doc.strip(), cut_all=False)
            doc = ' '.join(list(doc))
            predix = '__label__'
            label = predix + str(y[i])
            line = doc + '\t' + label + '\n'
            f.write(line)
            

In [41]:
# 第一层分类训练
x = np.array(train['ITEM_NAME'])
y = np.array(train['TYPE1'])
x_train, y_train, x_test, y_test = train_test_split(x, y)
data_write('train_type1.txt', x_train, y_train)
data_write('test_type1.txt', x_test, y_test)
classifier = fasttext.supervised('train_type1.txt', '1/0.model', label_prefix='__label__')
# classifier = fasttext.load_model('1/0.model.bin', label_prefix='__label__')
result = classifier.test('test_type1.txt')
precision_l1 = result.precision
recall_l2 = result.recall
length = len(x)
print('样本数量：%d; 精确率：%f; 回报率：%f' % (length, precision_l1, recall_l2))

样本数量：499447; 精确率：0.945221; 回报率：0.945221


In [43]:
# 第二层分类训练
precision2 = []
for i in range(len(type_dict)):
    x = np.array(train['ITEM_NAME'][type2_list[i]])
    y1 = np.array(train['TYPE2'][type2_list[i]])
    x_train, y1_train, x_test, y1_test = train_test_split(x, y1)
    data_write('train_type2.txt', x_train, y1_train)
    data_write('test_type2.txt', x_test, y1_test)
    classifier = fasttext.supervised('train_type2.txt', '2/'+str(i)+'.model', label_prefix='__label__')
    # classifier = fasttext.load_model('2/'+str(i)+'.model.bin', label_prefix='__label__')
    result = classifier.test('test_type2.txt')
    precision = result.precision
    precision2.append(precision)
    recall = result.recall
    length = len(x)
    print('样本数量：%d; 精确率：%f; 回报率：%f' % (length, precision, recall))

样本数量：350; 精确率：1.000000; 回报率：1.000000
样本数量：2268; 精确率：0.666667; 回报率：0.666667
样本数量：22679; 精确率：0.968254; 回报率：0.968254
样本数量：14096; 精确率：0.948936; 回报率：0.948936
样本数量：80869; 精确率：0.957962; 回报率：0.957962
样本数量：31014; 精确率：0.998066; 回报率：0.998066
样本数量：1556; 精确率：0.910256; 回报率：0.910256
样本数量：1853; 精确率：0.763441; 回报率：0.763441
样本数量：10316; 精确率：0.982558; 回报率：0.982558
样本数量：1824; 精确率：0.782609; 回报率：0.782609
样本数量：36953; 精确率：0.938853; 回报率：0.938853
样本数量：82800; 精确率：0.981159; 回报率：0.981159
样本数量：477; 精确率：0.500000; 回报率：0.500000
样本数量：12322; 精确率：0.893031; 回报率：0.893031
样本数量：34691; 精确率：0.978674; 回报率：0.978674
样本数量：31515; 精确率：0.619924; 回报率：0.619924
样本数量：23756; 精确率：0.930135; 回报率：0.930135
样本数量：3831; 精确率：0.859375; 回报率：0.859375
样本数量：3541; 精确率：0.516854; 回报率：0.516854
样本数量：35557; 精确率：0.961192; 回报率：0.961192
样本数量：44304; 精确率：0.992780; 回报率：0.992780
样本数量：22875; 精确率：0.902098; 回报率：0.902098


In [45]:
# 第三层分类训练
precision3 = []
for i in range(len(type2_dict)):
    x = np.array(train['ITEM_NAME'][type3_list[i]])
    y1 = np.array(train['TYPE3'][type3_list[i]])
    x_train, y1_train, x_test, y1_test = train_test_split(x, y1)
    data_write('train_type3.txt', x_train, y1_train)
    data_write('test_type3.txt', x_test, y1_test)
    classifier = fasttext.supervised('train_type3.txt', '3/'+str(i)+'.model', label_prefix='__label__')
    # classifier = fasttext.load_model('3/'+str(i)+'.model.bin', label_prefix='__label__')
    result = classifier.test('test_type3.txt')
    precision = result.precision
    precision3.append(precision)
    recall = result.recall
    length = len(x)
    print('样本数量：%d; 精确率：%f; 回报率：%f' % (length, precision, recall))

样本数量：350; 精确率：0.833333; 回报率：0.833333
样本数量：64; 精确率：0.500000; 回报率：0.500000
样本数量：198; 精确率：0.400000; 回报率：0.400000
样本数量：742; 精确率：0.868421; 回报率：0.868421
样本数量：286; 精确率：0.666667; 回报率：0.666667
样本数量：438; 精确率：0.363636; 回报率：0.363636
样本数量：232; 精确率：1.000000; 回报率：1.000000
样本数量：308; 精确率：0.125000; 回报率：0.125000
样本数量：2198; 精确率：0.945455; 回报率：0.945455
样本数量：7800; 精确率：0.876923; 回报率：0.876923
样本数量：5523; 精确率：0.920578; 回报率：0.920578
样本数量：357; 精确率：0.777778; 回报率：0.777778
样本数量：1009; 精确率：0.431373; 回报率：0.431373
样本数量：5792; 精确率：0.896552; 回报率：0.896552
样本数量：2695; 精确率：0.859259; 回报率：0.859259
样本数量：4025; 精确率：0.940594; 回报率：0.940594
样本数量：147; 精确率：1.000000; 回报率：1.000000
样本数量：2631; 精确率：0.871212; 回报率：0.871212
样本数量：606; 精确率：0.483871; 回报率：0.483871
样本数量：3992; 精确率：0.745000; 回报率：0.745000
样本数量：2618; 精确率：0.717557; 回报率：0.717557
样本数量：30367; 精确率：0.956550; 回报率：0.956550
样本数量：7031; 精确率：0.838068; 回报率：0.838068
样本数量：3453; 精确率：0.820809; 回报率：0.820809
样本数量：34773; 精确率：0.931570; 回报率：0.931570
样本数量：2627; 精确率：0.598485; 回报率：0.598485
样本数量：23068; 精确率：0.9835

In [63]:
# 第一层分类精确率
print('第一层精确率：%f' % (precision_l1))

# 第二层分类平均精确率计算
pre_2_aver = 0
for i in range(len(type_dict)):
    length = len(type2_list[i])
    pre_2_aver += length*1.0/len(train)*precision2[i]
print('第二层平均精确率：%f' % (pre_2_aver))

# 第三层分类平均精确率计算
pre_3_aver = 0
for i in range(len(type2_dict)):
    length = len(type3_list[i])
    pre_3_aver += length*1.0/len(train)*precision3[i]
print('第三层平均精确率：%f' % (pre_3_aver))

第一层精确率：0.945221
第二层平均精确率：0.934423
第三层平均精确率：0.866855


In [74]:
# 450万数据的分层预测(以0-1000样本为例)
test = pd.read_csv('test.tsv', sep='\t', encoding='gb18030')
pre_labels = []
classifier1 = fasttext.load_model('1/0.model.bin', label_prefix='__label__')
i = 0
for line in test['ITEM_NAME'][:1000]:
    line = jieba.cut(line.strip(), cut_all=False)
    line = [str(' '.join(list(line)))]
    
    label1 = classifier1.predict(line)[0][0]
    num1 = type_dict[label1]
    
    classifier2 = fasttext.load_model('2/'+str(num1)+'.model.bin', label_prefix='__label__')
    label2 = classifier2.predict(line)[0][0]
    num2 = type2_dict[label2]
    
    classifier3 = fasttext.load_model('3/'+str(num2)+'.model.bin', label_prefix='__label__')
    label3 = classifier3.predict(line)[0][0]
    
    label = str(label1) + '--' + str(label2) + '--' + str(label3)
    pre_labels.append(label)
    if(i%200==0):
        print(i)
    i = i+1

0
200
400
600
800


In [76]:
# 预测结果保存
pre_test = pd.DataFrame({'ITEM_NAME':test['ITEM_NAME'][:1000], 'PRE_LABEL':pre_labels})
pre_test.to_csv('predict_result/pre_test.csv', sep=',', header=True)

In [77]:
pre_test.head()

,ITEM_NAME,PRE_LABEL
0,腾讯QQ蓝钻贵族34个月直充,本地生活--游戏充值--QQ充值
1,腾讯QQ币24元24QQ币24个直充24Q币24个Q币24个QB/24QQB★自动充值,本地生活--游戏充值--游戏点卡
2,腾讯QQ币101元101QQ币101个直充101Q币101个Q币101个QB★自动充值,本地生活--游戏充值--游戏点卡
3,腾讯QQ币402元 402QB 402QQ币402个 402Q币 402个Q币 自动充值,本地生活--游戏充值--游戏点卡
4,腾讯QQ币380元 380QB 380QQ币380个 380Q币 380个Q币 自动充值,本地生活--游戏充值--游戏点卡
